In [ ]:
!pip install pandasql
!pip install SQLAlchemy==1.4.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandasql import sqldf

In [ ]:
def load_images_from_folder(folder):
    images = []
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):
                img = cv2.imread(filepath)
                if img is not None:
                  images.append([img,file,subdir.split('/')[-1]])
    return images

image_data = load_images_from_folder(r'/content/drive/MyDrive/KJSCE/LY/DL/ProjectData/ORIGINAL_CLEANED')

In [ ]:
df=pd.read_csv(r'/content/drive/MyDrive/KJSCE/LY/DL/ProjectData/cleaned_annotatation_data.csv')

In [ ]:
def get_pred_box(image_data):
  pred_boxes=[]
  image = image_data[0].copy()
  img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  cv2.rectangle(img, (4, 220) ,(57, 235), (255, 255, 255), -1)
  image=cv2.blur(img,(9,9))
  result = image.copy()
  lower = np.array([0,0,155])
  upper = np.array([179,150,255])
  mask = cv2.inRange(image, lower, upper)
  # cv2_imshow( mask)

  contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  contours=[cnt for cnt in contours if cv2.contourArea(cnt)>120]

  # result=image_data[0].copy()
  for cntr in contours:
      x,y,w,h = cv2.boundingRect(cntr)
      xmin,ymin,xmax,ymax = x,y,x+w,y+h
      cv2.rectangle(result, (x, y), (x+w, y+h), (0,0,0), 2)
      pred_boxes.append([xmin,ymin,xmax,ymax])
  # cv2_imshow(result)
  return pred_boxes


def get_ground_box(image_data):
  result = sqldf("select xmin,ymin,xmax,ymax from df where image='{}'".format(image_data[1]))
  ground_boxes=result.values.tolist()
  return ground_boxes

# MultiBox IOU Implementation

In [ ]:
def calcIOU(ground_boxes,pred_boxes,image=[1,2,3]):
  img1 = np.zeros((240,320,3), dtype = np.uint8)
  img2 = np.zeros((240,320,3), dtype = np.uint8)

  for x1,y1,x2,y2 in ground_boxes:
    ground = cv2.rectangle(img1, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 255), -1)

  for x1,y1,x2,y2 in pred_boxes:
    pred = cv2.rectangle(img2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 255), -1)

  # print('\n~~~~~~~~~~~~~~~~  GROUND  ~~~~~~~~~~~~~~~~\n')
  # cv2_imshow(ground)
  # print('\n~~~~~~~~~~~~~~  PREDICTED  ~~~~~~~~~~~~~~~\n')
  # cv2_imshow(pred)
  union = cv2.bitwise_or(ground,pred)
  intersection = cv2.bitwise_and(ground,pred)

  img_gray = cv2.cvtColor(union, cv2.COLOR_BGR2GRAY)
  contours, hierarchy = cv2.findContours(image=img_gray, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
  union_copy = union.copy()
  cv2.drawContours(image=union_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
  union_area = sum([cv2.contourArea(contour) for contour in contours])
  # print('\n~~~~~~~~~~~~~~~~  UNION  ~~~~~~~~~~~~~~~~\n')
  # cv2_imshow(union_copy)

  img_gray = cv2.cvtColor(intersection, cv2.COLOR_BGR2GRAY)
  contours, hierarchy = cv2.findContours(image=img_gray, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
  intersection_copy = intersection.copy()
  cv2.drawContours(image=intersection_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
  intersection_area = sum([cv2.contourArea(contour) for contour in contours])
  # print('\n~~~~~~~~~~~~~  INTERSECTION  ~~~~~~~~~~~~~\n')
  # cv2_imshow(intersection_copy)

  # print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')
  # print('Intersection Area:',intersection_area)
  # print('Union Area:',union_area)
  IOU = round(intersection_area/union_area,5)
  # print('IOU Score: ',IOU)
  return [intersection_area,union_area,IOU,image[1],image[2]]

In [ ]:
ground_boxes = [[29.01960784, 128.2352941, 297.254902, 153.3333333],[278.0392157, 163.1372549, 312.5490196, 183.5294118]]
pred_boxes = [[32.01960784, 128.2352941, 295.254902, 150.3333333],[278.0392157, 163.1372549, 312.5490196, 183.5294118]]
calcIOU(ground_boxes,pred_boxes)

[6466.0, 7380.0, 0.87615, 2, 3]

In [ ]:
iou_map = []
for image in image_data:
  try:
    ground_boxes = get_ground_box(image)
    pred_boxes = get_pred_box(image)
    data=calcIOU(ground_boxes,pred_boxes,image)
    iou_map.append(data)
  except Exception as e:
    print(image[1:],ground_boxes,pred_boxes)
    pass

In [ ]:
iou_map_df = pd.DataFrame(iou_map, columns=['intersection_area','union_area','IOU','name','label'])
average_label_iou = iou_map_df.groupby('label')['IOU'].mean()
average_iou = iou_map_df['IOU'].mean()

print(f'Average IOU: {average_iou}\n')
print(average_label_iou)

Average IOU: 0.36930981308411215

label
PID_thermal       0.296024
diode_thermal     0.377565
dust_thermal      0.069382
multi_thermal     0.336140
single_thermal    0.436797
Name: IOU, dtype: float64


## TEST

In [ ]:
#replace FLIR with avg of surroundings
def get_pred_boxes1(image1,result1):
  pred_boxes=[]
  image=image1[0]
  result=result1[0]
  x, y, w, h = 306,29,9,19
  roi = image[y:y+h, x:x+w].copy()
  mask = np.zeros_like(image[y:y+h, x:x+w])
  image[y:y+h, x:x+w] = mask
  roi[0, :] = [255, 255, 255]
  roi[-1, :] = [255, 255, 255]
  roi[:, 0] = [255, 255, 255]
  roi[:, -1] = [255, 255, 255]
  xywh_list = [[5,5,71,19],[227,5,39,18],[279,215,37,20]]
  for x,y,w,h in xywh_list:
    image[y:y+h, x:x+w][np.all(image[y:y+h, x:x+w] > [230, 230, 230], axis=-1)] = [0, 0, 0]

  x,y,w,h =4,219,52,18
  region = image[y:y+h, x:x+w]
  mean_pixel_value = region[np.any(region<[220,220,220], axis=-1)].mean(axis=0)
  region[np.all(region>[220,220,220], axis=-1)] = mean_pixel_value


  min_pixel = np.array(roi.min(axis=(0, 1)))
  max_pixel = np.array(roi.max(axis=(0, 1)))

  print(min_pixel,max_pixel)


  image=cv2.blur(image,(7,7))
  mask = cv2.inRange(image, min_pixel, max_pixel)
  # cv2_imshow(mask)


  contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  contours=[cnt for cnt in contours if cv2.contourArea(cnt)>50]
  # cv2_imshow(cv2.drawContours(image, contours, -1, (0,255,0), 2))
  print(len(contours))


  for cntr in contours:
      x,y,w,h = cv2.boundingRect(cntr)
      xmin,ymin,xmax,ymax = x,y,x+w,y+h
      cv2.rectangle(result, (x, y), (x+w, y+h), (0,0,0), 2)
      pred_boxes.append([xmin,ymin,xmax,ymax])

  return pred_boxes
  # cv2_imshow(result)

In [ ]:
image_data = load_images_from_folder(r'/content/drive/MyDrive/KJSCE/LY/DL/ProjectData/ORIGINAL_CLEANED/multi_thermal')
iou_map = []
for image in image_data:
  try:
    ground_boxes = get_ground_box(image)
    pred_boxes = get_pred_boxes1(image,image)
    data=calcIOU(ground_boxes,pred_boxes,image)
    iou_map.append(data)
  except Exception as e:
    print(e)
    print(image[1:],ground_boxes,pred_boxes)
    pass

In [ ]:
iou_map_df = pd.DataFrame(iou_map, columns=['intersection_area','union_area','IOU','name','label'])
average_label_iou = iou_map_df.groupby('label')['IOU'].mean()
# average_iou = iou_map_df['IOU'].mean()

# print(f'Average IOU: {average_iou}\n')
print(average_label_iou)

label
multi_thermal    0.193492
Name: IOU, dtype: float64


# SINGLE BOX IOU

In [ ]:
box1 = 270.0392157,140.8235294,305.4509804,163.6078431
box2 = 272.0392157,145.8235294,305.4509804,163.6078431

In [ ]:
x1, y1, x2, y2 = box1
x3, y3, x4, y4 = box2
x_inter1 = int(max(x1, x3))
y_inter1 = int(max(y1, y3))
x_inter2 = int(min(x2, x4))
y_inter2 = int(min(y2, y4))
width_inter = abs(x_inter2 - x_inter1)
height_inter = abs(y_inter2 - y_inter1)
area_inter = width_inter*height_inter
width_box1 = abs(x2-x1)
height_box1 = abs(y2 - y1)
width_box2 = abs(x4 - x3)
height_box2 = abs(y4 - y3)
area_box1 = width_box1*height_box1
area_box2 = width_box2*height_box2
area_union = area_box1 + area_box2 - area_inter
iou = area_inter / area_union
print(round(iou,5))

0.73602


In [ ]:
print(area_inter)

594


In [ ]:
print(area_union)

807.0380602907712
